In [1]:
import gym
import random
import tflearn as tf
import numpy as np
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
LR = 1e-3
env = gym.make('CartPole-v0')
env.reset()
max_goal= 500
score_requirement= 50     #change to adjust qualtity of training data
initial_games = 10000


/home/ankit/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
def some_random_games():
    for episode in range(100):
        env.reset()
        for t in range(max_goal):
            env.render()
            action = env.action_space.sample()     #  provided in gym
            observation, reward, done, info = env.step(action)
            if done:
                break

In [4]:
# some_random_games()

In [4]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = [] 
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(max_goal):
#             env.render()
            action = random.randrange(0,2)
    
            observation, reward, done, info = env.step(action)
            if len(prev_observation)>0:
                game_memory.append([prev_observation,action])

            prev_observation = observation
            score+= reward
            if done:
                break
        if score>=score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1]==1:
                    output = [0,1]
                elif data[1] ==0:
                    output = [1,0]
                training_data.append([data[0],output])
                
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
#     np.save('saved_score100.npy', training_data_save)
    print('Average accepted score:', mean(accepted_scores))
    print('Median accepted score :', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [5]:
x=initial_population()

Average accepted score: 61.45478036175711
Median accepted score : 58.0
Counter({50.0: 35, 53.0: 28, 51.0: 27, 57.0: 25, 54.0: 23, 58.0: 22, 59.0: 21, 52.0: 21, 55.0: 14, 67.0: 13, 64.0: 11, 66.0: 11, 61.0: 10, 60.0: 9, 63.0: 9, 56.0: 9, 62.0: 8, 69.0: 8, 70.0: 8, 71.0: 7, 72.0: 6, 65.0: 6, 73.0: 6, 68.0: 5, 75.0: 4, 77.0: 4, 83.0: 3, 90.0: 3, 85.0: 3, 82.0: 3, 79.0: 2, 86.0: 2, 101.0: 2, 81.0: 2, 74.0: 2, 99.0: 1, 80.0: 1, 84.0: 1, 102.0: 1, 117.0: 1, 115.0: 1, 76.0: 1, 92.0: 1, 88.0: 1, 100.0: 1, 78.0: 1, 89.0: 1, 121.0: 1, 96.0: 1, 94.0: 1})


In [6]:
x

[[array([-0.01106154, -0.15926296,  0.00614179,  0.28497652]), [1, 0]],
 [array([-0.0142468 , -0.35447197,  0.01184132,  0.57959018]), [0, 1]],
 [array([-0.02133624, -0.15951794,  0.02343312,  0.29066091]), [1, 0]],
 [array([-0.0245266 , -0.35496605,  0.02924634,  0.5906413 ]), [0, 1]],
 [array([-0.03162592, -0.16026553,  0.04105917,  0.3073126 ]), [1, 0]],
 [array([-0.03483123, -0.35594776,  0.04720542,  0.61265657]), [1, 0]],
 [array([-0.04195018, -0.55169653,  0.05945855,  0.91982587]), [0, 1]],
 [array([-0.05298411, -0.35742644,  0.07785507,  0.6464062 ]), [0, 1]],
 [array([-0.06013264, -0.16347068,  0.09078319,  0.37922013]), [1, 0]],
 [array([-0.06340206, -0.35975669,  0.09836759,  0.69909031]), [0, 1]],
 [array([-0.07059719, -0.16612621,  0.1123494 ,  0.43892212]), [0, 1]],
 [array([-0.07391971,  0.02724124,  0.12112784,  0.18366087]), [0, 1]],
 [array([-0.07337489,  0.22044056,  0.12480106, -0.06849018]), [1, 0]],
 [array([-0.06896608,  0.02377085,  0.12343126,  0.26081631]), [

In [3]:
def neural_network_model(input_size):
    network = input_data(shape=[None,input_size,1],name  = 'input')
    
    network = fully_connected(network,128,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,256,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,512,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,256,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = 'relu')
    network = dropout(network,0.8)
    
    network = fully_connected(network,2,activation = 'softmax')
    network = regression(network,optimizer = 'adam', learning_rate=LR, 
                         loss='categorical_crossentropy',name='targets')
    
    model =  tf.DNN(network,tensorboard_dir= 'Final_log_good_training_data')
    
    return model

def train_model(training_data, model  = False):
    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]
    
    if not model:
        model =neural_network_model(input_size= len(X[0]))
        
    model.fit({'input':X},{'targets':y},n_epoch=4,snapshot_step=500,show_metric =True,run_id = 'openai')
    
    return model
    
    
    
    

In [4]:
training_data = np.load('saved.npy')
# training_data = initial_population()

model= train_model(training_data)

Training Step: 4599  | total loss: 0.67719 | time: 20.220s
| Adam | epoch: 004 | loss: 0.67719 - acc: 0.5870 -- iter: 73536/73539
Training Step: 4600  | total loss: 0.67948 | time: 20.238s
| Adam | epoch: 004 | loss: 0.67948 - acc: 0.5768 -- iter: 73539/73539
--


In [ ]:
# model.save('cartpole0.model')

In [ ]:
# training_data = np.load('saved.npy')
# X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
# model =neural_network_model(input_size= len(X[0]))

# model = model.load('cartpole0.model')

In [7]:
scores=[]
choices =[]

for each_game in range(100):
    score=0
    game_memory=[]
    prev_obs = []
    env.reset()
    for _ in range(max_goal):
        env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])
        choices.append(action)
        
        new_observation,reward,done,info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation,action])
        score += reward
        if done:
            break
    scores.append(score)
print('Average Score', sum(scores)/len(scores))
print('Choices1:{}','Choice 2: {}'.format(choices.count(1)/len(choices)),choices.count(0)/len(choices))





Average Score 168.07
Choices1:{} Choice 2: 0.5105015767239841 0.48949842327601595


In [ ]:
# model.save('cartpole0BestScore.model')

In [ ]:
# env.render()


In [ ]:
# sorted_scores=scores.sort()

In [ ]:
final_scores=scores

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.sort(final_scores))
plt.title('Default Cartpole')
plt.xlabel("episode")
plt.ylabel('average score')

In [ ]:
np.save('savev1raw0.npy', final_scores)


In [ ]:
score_model1 = np.load('savev1raw0.npy')
score_model2 = np.load('savev1raw.npy')
score_model3 = np.load('savev1raw1.npy')

In [ ]:
plt.figure(figsize=(12,5))

plt.plot(np.sort(score_model1),'b')
plt.plot(np.sort(score_model2),'g')
plt.plot(np.sort(score_model3),'r')
plt.title('Default Cartpole')
plt.xlabel("episode")
plt.ylabel('average score')
plt.legend(['basic cartpole', 'cartpole new env', 'cartpole new ev +TL'])

In [ ]:
plt.figure(figsize=(12,5))
x = ['basic cartpole', 'cartpole new env', 'cartpole new ev +TL']
plt.bar(x[0],np.average(score_model1),width=0.6)
plt.bar(x[1],np.average(score_model2),width=0.6)
plt.bar(x[2],np.average(score_model3),width=0.6)